# Short example script for calculating the dimensionless linear power spectrum

$$\Delta(k_\rm{p},z=1.0) = k_\rm{p}^3 P_{\rm{lin}}(k_\rm{p},z=1.0)$$ at pivot scale $k_\rm{p} = 0.2\, h/\rm{Mpc}$

## Import modules, set up settings

In [1]:
import classy
from classy import Class
import numpy as np

#pivot scale + redshift
z_p = 1.0
k_p = 0.2 #our pivot scale, in h/Mpc units

#settings to configure CLASS computations
settings = {
    'output': 'mPk', #which quantities we want CLASS to compute
    'z_pk' : z_p,
    'non linear' : 'none',
    'P_k_max_1/Mpc':1.0 # CLASS uses 1/Mpc units
}


## Inspect CLASS version

NOTE: For Beyond-2pt submissions, should be ```v.2.9.4```

In [2]:
print(f"CLASS version: {classy.__version__}")

CLASS version: v2.9.4


## Compute ```norm``` == dimensionless linear matter power spectrum at $k_p$

### Define function(s)

In [3]:
def compute_norm(params):
    #initialize Class instance and set parameters
    cosmo = Class();
    cosmo.set(settings);
    cosmo.set(params)
    cosmo.compute()

    #if CLASS used shooting method to determine A_s, check that it worked to better that 0.1% in sigma8
    if 'sigma8' in params:
        delta_sigma8 = np.abs((params['sigma8']-cosmo.sigma(8/cosmo.h(),0))/params['sigma8'])
        assert delta_sigma8 < 1.e-3, f"A_s shooting method caused %.4f fractional bias in sigma8"%(delta_sigma8)

    h = cosmo.h()
    k_p_CLASS = k_p*h # convert to 1/Mpc units used by CLASS
    norm = k_p**3*cosmo.pk(k_p_CLASS,z_p)*h**3 #factor h**3 to convert P(k) from 1/Mpc to h/Mpc units
    return norm

### Test CLASS with fiducial cosmology

Test whether we recover the same ```norm``` when specifying either $\sigma_8$ or $A_s$.
Really should be the same for linear $P_L(k)$ + standard cosmology.

In [4]:
test_params_sigma8 = {
    'omega_b': 0.0222554,
    'omega_cdm': 0.118967,
    'sigma8': 0.816,
    #'A_s': 2.1e-09,
    'n_s': 0.991533,
    'N_ur': 2.0328,
    'N_ncdm' : 1.0,
    'omega_ncdm': 0.00064420,
    'recombination' : 'HyRec',
    'tau_reio' : 0.0544
}

norm_sigma8 = compute_norm(test_params_sigma8)

test_params_As = {
    'omega_b': 0.0222554,
    'omega_cdm': 0.118967,
    #'sigma8': 0.84,
    'A_s': 2.1e-09,
    'n_s': 0.991533,
    'N_ur': 2.0328,
    'N_ncdm' : 1.0,
    'omega_ncdm': 0.00064420,
    'recombination' : 'HyRec',
    'tau_reio' : 0.0544
}
norm_As = compute_norm(test_params_As)

print("(norm_As==norm_sigma8)?")
print(np.isclose(norm_sigma8,norm_As,rtol=1e-05, atol=1e-14))

(norm_As==norm_sigma8)?
True


### Compute ```norm``` and $A_s$, then add their columns to the new output files

This could be done much faster with bash scripts. Here I'm just trying to make it humanly readable using ```python```.

In [5]:
# Specify the input and output file paths

chain_param_path='/cobra/ptmp/minh/lefty_challenge/flatLCDM/box1/sampling_bO/Eulerian_NUFFT/twolptthird_nufft/kmax015/'
chain_prefix=['chain1_fixed_highsigma/sampling_bO_2lptthird_nufft_FOURIER_NGEul1024_L0.21_cube_logP_params','chain1/sampling_bO_2lptthird_nufft_FOURIER_NGEul1024_L0.21_cube_logP_params']

input_files = []
output_files = []
for chain in chain_prefix:
    input_files.append(chain_param_path+chain+'.1.txt')
    output_files.append(chain_param_path+chain+'_include_norm_As.1.txt')

# Specify the *fixed* fiducial cosmological parameters as a dict

params = {
    'omega_b': 0.0222554,
    'omega_cdm': 0.118967,
    'n_s': 0.991533,
    'N_ur': 2.0328,
    'N_ncdm' : 1.0,
    'omega_ncdm': 0.00064420,
    'recombination' : 'HyRec',
    'tau_reio' : 0.0544
}
    
# Specify fiducial value of As
# This fiducial value was used to generate the fiducial input linear P(k) for LEFTfield
# LEFTfield only infers alpha = As/As_fid
As_fid = 2.1e-9 ## Planck 2018, TT,TE,EE+lowE+lensing; Table 2 of arXiv:1807.06209

# Open input and output files
for (input_file,output_file) in zip(input_files,output_files):
    with open(input_file, 'r') as file:
        with open(output_file, 'w') as output:
            for line in file:
                columns = line.split()
                if line.startswith('#'):
                    columns.insert(3, 'norm')
                    columns.insert(4, 'A_s')
                    output.write('\t'.join(columns) + '\n')
                else:
                    As = float(columns[2]) * As_fid
                    params['A_s'] = As
                    norm = compute_norm(params)
                    # Insert the product as a new column between column 2 and 3
                    columns.insert(3, str(norm))
                    columns.insert(4, str(As))
                    # Write the modified line to the output file
                output.write('\t'.join(columns) + '\n')

print("File processing complete. Outputs are save at:")
print(output_files)

File processing complete. Outputs are save at:
['/cobra/ptmp/minh/lefty_challenge/flatLCDM/box1/sampling_bO/Eulerian_NUFFT/twolptthird_nufft/kmax015/chain1_fixed_highsigma/sampling_bO_2lptthird_nufft_FOURIER_NGEul1024_L0.21_cube_logP_params_include_norm_As.1.txt', '/cobra/ptmp/minh/lefty_challenge/flatLCDM/box1/sampling_bO/Eulerian_NUFFT/twolptthird_nufft/kmax015/chain1/sampling_bO_2lptthird_nufft_FOURIER_NGEul1024_L0.21_cube_logP_params_include_norm_As.1.txt']


### Check format of output files

In [7]:
out_norm = np.loadtxt(output_files[0],usecols=2)
out_As = np.loadtxt(output_files[0],usecols=3)
print("Number of raw MCMC samples:")
print(out_norm.shape[0])
print("Unweighted mean norm:")
print(out_norm.mean())
print("Unweighted mean As:")
print(out_As.mean())
    

Number of raw MCMC samples:
8430
Unweighted mean norm:
5.282092100615641
Unweighted mean As:
1.8875540450814366e-09
